# IA Construire une application IA financière avec du prompt engineering
Ce Chapitre couvre :
1) Rédiger un prompt efficace pour l’analyse financière  
2) Tester le prompt sur un **tableau** (KPIs calculés)  
3) Créer un premier script Python avec l’API OpenAI (réponse texte)  
4) Transformer un prompt en **outil réutilisable**  
5) Créer une **interface Gradio** pour tester plusieurs prompts



#  Mini-Application d’Analyse Financière avec IA

Construire une application qui :
1. **Récupère automatiquement** les données de marché depuis Yahoo Finance.
2. **Calcule des KPIs financiers standards** (dernier prix, rendements 1M & 3M, volatilité annualisée).
3. **Formate les résultats** pour être injectés dans un prompt.
4. **Demande à une IA** (OpenAI API) de produire une **analyse synthétique** : résumé, KPIs, risques, opportunités et actions recommandées.
5. Fournit une **sortie texte claire et structurée** pour un décideur non spécialiste.


## 1) Rédiger un prompt efficace pour l’analyse financière

**Objectif** : construire des prompts **clairs** et **contextualisés** pour analyser des **données financières**.  
**Idées clés** :
- **Rôle & contexte** : précisez le rôle de l’IA (ex. *analyste financier senior*) et le contexte (secteur, période, type de données).  
- **Tâche** : décrivez exactement ce que vous voulez (résumé, KPIs, risques, recommandations).  
- **Contraintes** : ton professionnel, concision, citer les chiffres avec unité/devise, éviter le jargon inutile.  
- **Format de sortie** : imposez **JSON** (clé/valeurs) pour exploiter facilement la réponse dans du code.


In [8]:
PROMPT_TEMPLATE = """Tu es analyste financier senior.

CONTEXTE
{context}

DONNÉES FOURNIES (Yahoo Finance)
{data_desc}

OBJECTIF
Produis une analyse courte, actionnable et strictement basée sur les données ci-dessus, pour un décideur non spécialiste.

RAPPELS IMPORTANTS
- Langue : français. Ton professionnel, phrases courtes.
- N’invente rien : si une information manque, écris exactement "unknown".
- Cite systématiquement les chiffres avec unité/devise et période (ex. 185,42 USD, 3,20 %, 1M, 3M, volatilité annualisée 20j).
- Formate les nombres avec 2 décimales quand c’est utile.
- Ne produis PAS de JSON, de code, ni de balises. Sortie en TEXTE uniquement avec TITRES et LISTES À PUCES.
- Longueur maximale par section : 4–6 puces ou 5 lignes selon le cas.

FORMAT DE SORTIE EXACT (RESPECTE LES TITRES CI-DESSOUS)

Résumé (5 lignes max)
- [1 phrase d’ouverture sur la tendance générale du portefeuille]
- [1–2 faits clés chiffrés, ex. meilleur/moins bon ticker avec chiffre et période]
- [1 phrase sur la volatilité globale ou la dispersion des rendements]
- [1 phrase de conclusion courte]

KPIs clés (3–5 puces, chiffres exacts)
- [Ticker] Dernier cours : [valeur + devise] (latest)
- [Ticker] Rendement 1M : [valeur %] (1M)
- [Ticker] Rendement 3M : [valeur %] (3M)
- [Ticker] Volatilité annualisée 20j : [valeur %]
- Mets "unknown" si un KPI manque.

Risques (2–3 puces, 1 ligne chacun, appuyés par un chiffre)
- [Risque synthétique] — évidence : [ticker/indicateur = valeur + période]

Opportunités (2–3 puces, 1 ligne chacun, appuyées par un chiffre)
- [Opportunité synthétique] — évidence : [ticker/indicateur = valeur + période]

Actions proposées (3 puces, concrètes et exécutables par un non spécialiste)
- [Action #1 courte] — fondée sur [KPI/ticker + chiffre]
- [Action #2 courte] — fondée sur [KPI/ticker + chiffre]
- [Action #3 courte] — fondée sur [KPI/ticker + chiffre]

Notes sur les données (optionnel, 2 puces max)
- [Ex. données manquantes ou irrégularités détectées : "unknown" pour ...]
- [Ex. rappel de périmètre ou limites : ex. univers restreint à 4 tickers]

CONTRAINTE FINALE
- Réponds UNIQUEMENT en TEXTE selon la structure ci-dessus. AUCUN JSON, AUCUN CODE, AUCUNE BALISE.
"""

PROMPT_TEMPLATE


'Tu es analyste financier senior.\n\nCONTEXTE\n{context}\n\nDONNÉES FOURNIES (Yahoo Finance)\n{data_desc}\n\nOBJECTIF\nProduis une analyse courte, actionnable et strictement basée sur les données ci-dessus, pour un décideur non spécialiste.\n\nRAPPELS IMPORTANTS\n- Langue : français. Ton professionnel, phrases courtes.\n- N’invente rien : si une information manque, écris exactement "unknown".\n- Cite systématiquement les chiffres avec unité/devise et période (ex. 185,42 USD, 3,20 %, 1M, 3M, volatilité annualisée 20j).\n- Formate les nombres avec 2 décimales quand c’est utile.\n- Ne produis PAS de JSON, de code, ni de balises. Sortie en TEXTE uniquement avec TITRES et LISTES À PUCES.\n- Longueur maximale par section : 4–6 puces ou 5 lignes selon le cas.\n\nFORMAT DE SORTIE EXACT (RESPECTE LES TITRES CI-DESSOUS)\n\nRésumé (5 lignes max)\n- [1 phrase d’ouverture sur la tendance générale du portefeuille]\n- [1–2 faits clés chiffrés, ex. meilleur/moins bon ticker avec chiffre et période]\n-

# Préparation et importations

In [9]:
!pip install yfinance openai pandas numpy matplotlib gradio

In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
# ---------------------- Import Yahoo Finance (Adj Close) ----------------------
UNIVERSE   = ["AAPL","MSFT","JPM","XOM"]   # modifiez librement
START_DATE = "2020-01-01"
END_DATE   = None  # jusqu'a aujourd'hui

# Telechargement des donnees (on suit exactement votre snippet)
data = yf.download(UNIVERSE, start=START_DATE, end=END_DATE, progress=False, auto_adjust=False)

# Si plusieurs tickers, on prend la colonne 'Adj Close'
if isinstance(data.columns, pd.MultiIndex):
    prices = data["Adj Close"].copy()           # colonnes = tickers
else:
    prices = data[["Adj Close"]].copy()         # colonnes = ["Adj Close"]
    if isinstance(UNIVERSE, list) and len(UNIVERSE) == 1:
        prices.columns = [UNIVERSE[0]]          # renommer la colonne unique en nom du ticker

# Apercu brut
print("Apercu du tableau de prix (Adj Close) :")
print(prices.head())
# ------------------------------------------------------------------------------

# KPIs sur la base des prix
rets = np.log(prices).diff()
win = 20  # fenetre pour volatilite realisee (approx. 1 mois de bourse)

kpis = []
for col in prices.columns:
    s = prices[col].dropna()
    r = rets[col].dropna()
    last_price = float(s.iloc[-1]) if len(s) else np.nan
    r_1m = float(np.exp(r.iloc[-21:].sum()) - 1) if len(r) >= 21 else np.nan
    r_3m = float(np.exp(r.iloc[-63:].sum()) - 1) if len(r) >= 63 else np.nan
    vol20 = float(r.iloc[-win:].std() * np.sqrt(252)) if len(r) >= win else np.nan
    kpis.append({
        "ticker": col,
        "last_price": last_price,
        "return_1m": r_1m,
        "return_3m": r_3m,
        "vol20_annualized": vol20
    })

df_kpis = pd.DataFrame(kpis)
display(df_kpis)

# Formatter propre pour injection dans le prompt
k = df_kpis.copy()
k["last_price"]        = k["last_price"].map(lambda x: f"{x:,.2f} USD" if pd.notnull(x) else "NA")
k["return_1m"]         = k["return_1m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["return_3m"]         = k["return_3m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["vol20_annualized"]  = k["vol20_annualized"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")

DATA_DESC_YF = "KPIs (Yahoo Finance):\n" + k.to_string(index=False)
print("\n=== Description pour le prompt ===\n", DATA_DESC_YF)


Apercu du tableau de prix (Adj Close) :
Ticker           AAPL         JPM        MSFT        XOM
Date                                                    
2020-01-02  72.538513  120.154724  152.791122  54.131073
2020-01-03  71.833290  118.569092  150.888641  53.695889
2020-01-06  72.405678  118.474823  151.278625  54.108173
2020-01-07  72.065163  116.460648  149.899261  53.665356
2020-01-08  73.224411  117.369171  152.286926  52.856052


,ticker,last_price,return_1m,return_3m,vol20_annualized
0,AAPL,252.309998,0.110715,0.253207,0.285211
1,JPM,313.420013,0.062801,0.108674,0.165154
2,MSFT,510.149994,0.011680,0.038025,0.162330
3,XOM,114.559998,0.025237,0.066956,0.201705



=== Description pour le prompt ===
 KPIs (Yahoo Finance):
ticker last_price return_1m return_3m vol20_annualized
  AAPL 252.31 USD   11.07 %   25.32 %          28.52 %
   JPM 313.42 USD    6.28 %   10.87 %          16.52 %
  MSFT 510.15 USD    1.17 %    3.80 %          16.23 %
   XOM 114.56 USD    2.52 %    6.70 %          20.17 %


In [11]:
# Imports communs
import os, json, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['axes.grid'] = True

# Détection  de la clé API
api_key = os.getenv("OPENAI_API_KEY", "")
print("OPENAI_API_KEY défini :", api_key[:4])

OPENAI_API_KEY défini : sk-p


In [12]:

from openai import OpenAI

client = OpenAI(api_key=api_key)

In [13]:
CONTEXT = "Portefeuille multi-secteurs, horizon court terme."
prompt_yf = PROMPT_TEMPLATE.format(context=CONTEXT, data_desc=DATA_DESC_YF)

resp_yf = client.chat.completions.create(
    model="gpt-5",
    messages=[{"role":"user","content": prompt_yf}]
)
print(resp_yf.choices[0].message.content)


Résumé (5 lignes max)
- L’univers suivi affiche une dynamique positive à court terme.
- Meilleur 1M: AAPL à 11,07 % (1M); Meilleur 3M: AAPL à 25,32 % (3M).
- Moins bon: MSFT à 1,17 % (1M) et 3,80 % (3M).
- Volatilité dispersée: de 16,23 % à 28,52 % (volatilité annualisée 20j).
- Priorité au momentum (AAPL, JPM); surveiller MSFT.

KPIs clés (3–5 puces, chiffres exacts)
- AAPL Dernier cours : 252,31 USD (latest); Rendement 1M : 11,07 % (1M); Rendement 3M : 25,32 % (3M); Volatilité annualisée 20j : 28,52 %.
- JPM Dernier cours : 313,42 USD (latest); Rendement 1M : 6,28 % (1M); Rendement 3M : 10,87 % (3M); Volatilité annualisée 20j : 16,52 %.
- MSFT Dernier cours : 510,15 USD (latest); Rendement 1M : 1,17 % (1M); Rendement 3M : 3,80 % (3M); Volatilité annualisée 20j : 16,23 %.
- XOM Dernier cours : 114,56 USD (latest); Rendement 1M : 2,52 % (1M); Rendement 3M : 6,70 % (3M); Volatilité annualisée 20j : 20,17 %.

Risques (2–3 puces, 1 ligne chacun, appuyés par un chiffre)
- Volatilité élevée

In [15]:
PROMPT_TEMPLATE = """Tu es analyste financier senior.

CONTEXTE
{context}

DONNÉES (Yahoo Finance)
{data_desc}

OBJECTIF
Rédige une analyse financière claire, fluide et actionnable basée uniquement sur les données fournies. 
Ton destinataire est un décideur non spécialiste : il attend un texte compréhensible, avec des phrases courtes mais informatives.

CONSIGNES D’ÉCRITURE
- Langue : français. Style professionnel, fluide et direct.
- Écris en paragraphes, pas en puces ni en JSON.
- N’invente rien : si une donnée est manquante, mentionne "inconnu".
- Cite toujours les chiffres avec unité/devise et période (ex. 185,42 USD, 3,20 %, 1M, 3M, volatilité annualisée 20j).
- Mets en avant les faits chiffrés pour appuyer chaque analyse.
- Structure ton texte en **sections avec titres** (Résumé, KPIs clés, Risques, Opportunités, Recommandations).
- Chaque section doit contenir 3–6 phrases rédigées.

FORMAT DE SORTIE
1. **Résumé** : 1 paragraphe qui décrit la tendance générale et les faits marquants.
2. **KPIs clés** : 1 paragraphe qui explique les indicateurs principaux (dernier cours, rendements, volatilité).
3. **Risques** : 1 paragraphe qui identifie 2–3 risques, avec preuves chiffrées.
4. **Opportunités** : 1 paragraphe qui met en avant 2–3 points positifs, avec preuves chiffrées.
5. **Recommandations** : 1 paragraphe qui propose 2–3 actions claires et adaptées à un non spécialiste.

CONTRAINTE FINALE
- Réponds uniquement en TEXTE rédigé, structuré en paragraphes avec titres.
- Pas de JSON, pas de code, pas de listes à puces.
"""


In [16]:
def run_prompt(context: str, data_desc: str, extra: str = "", model: str = "gpt-5-mini") -> str:
    """Construit le prompt et renvoie la reponse TEXTE (pas de JSON)."""
    prompt_here = PROMPT_TEMPLATE.format(context=context, data_desc=data_desc)
    if extra:
        prompt_here += "\n\nConsignes additionnelles :\n" + extra
    r = client.chat.completions.create(
        model=model,
        messages=[{"role":"user","content": prompt_here}]
    )
    return r.choices[0].message.content

# Exemple rapide : rejouer sur les KPIs Yahoo
print(run_prompt(CONTEXT, DATA_DESC_YF, extra="Souligne les divergences sectorielles et propose 3 actions."))


Résumé
Le portefeuille multi-secteurs montre une dynamique court terme marquée par une forte surperformance d’AAPL et des performances plus modérées pour JPM, MSFT et XOM. Apple affiche un rendement 3M de 25,32 % et un rendement 1M de 11,07 % tandis que Microsoft reste plus stable avec 3,80 % sur 3M et 1,17 % sur 1M. Les financières (JPM) et l’énergie (XOM) enregistrent des gains intermédiaires avec respectivement 10,87 % et 6,70 % sur 3M. Les niveaux de volatilité diffèrent sensiblement entre titres, ce qui influence le profil risque/rendement du portefeuille.

KPIs clés
AAPL dernier cours 252,31 USD, rendement 1M 11,07 %, rendement 3M 25,32 %, volatilité annualisée 20j 28,52 %. JPM dernier cours 313,42 USD, rendement 1M 6,28 %, rendement 3M 10,87 %, volatilité annualisée 20j 16,52 %. MSFT dernier cours 510,15 USD, rendement 1M 1,17 %, rendement 3M 3,80 %, volatilité annualisée 20j 16,23 %. XOM dernier cours 114,56 USD, rendement 1M 2,52 %, rendement 3M 6,70 %, volatilité annualisée 2

## Défi — Refaire l’application **100% local** avec Ollama

**Objectif.** Reprendre votre mini-application (KPIs Yahoo Finance → prompt engineering → réponse texte) et la refaire **sans aucun appel cloud**, en utilisant **uniquement Ollama** en local.

### Contraintes
- **Modèle** : un LLM open-source lancé via Ollama (ex. `llama3:8b`, `mistral:7b-instruct`, `qwen2.5:7b-instruct`).
- **Package** : utiliser le package Python `ollama` (`pip install ollama`), pas l’API OpenAI.
- **Données** : récupérer les prix avec **Yahoo Finance** (`yfinance`), calculer au minimum : **dernier cours**, **rendement 1 mois**, **rendement 3 mois**, **volatilité 20j annualisée**.
- **Prompt** : réutiliser le **template en sortie texte** (titres + puces, **pas de JSON**), en injectant votre tableau de KPIs formaté.
- **Sortie** : texte clair en français (résumé ≤ 5 lignes, 3 KPIs en puces, 2 risques en puces, 3 actions en puces).

### À faire (checklist)
1. **Installer** et démarrer Ollama, puis tirer un modèle :  
   `ollama pull llama3:8b`
2. **Calculer les KPIs** (Yahoo Finance) et construire un texte `data_desc` lisible (tableau formaté).
3. **Construire le prompt** (même structure que précédemment : contexte + `data_desc` + consignes).
4. **Générer localement** avec `ollama.chat` (sortie **texte** uniquement, pas de JSON).
5. **Comparer** rapidement la réponse locale vs la version cloud : clarté, précision chiffrée, ton.

### Livrables attendus
- Le **script/notebook** complet (import YF, KPIs, prompt, appel `ollama.chat`).
- Le **prompt final** injecté (copié en clair).
- La **réponse texte** du modèle (résumé, 3 KPIs, 2 risques, 3 actions).
- 3–5 lignes de **retour d’expérience** (vitesse, qualité, limites, pistes d’amélioration : quantisation, autre modèle, séparation par secteurs…).

> Astuce : si le modèle « oublie » les chiffres, **raccourcissez** le tableau (top 4–6 tickers), **réitérez** la consigne « citer les chiffres avec unité/devise » et baissez la **température** (≈ 0.2).


# LLM local
ollama pull llama3:8b
# Librairies Python
pip install ollama yfinance pandas numpy matplotlib


In [6]:
%pip install ollama yfinance pandas numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [7]:
!ollama list



NAME                  ID              SIZE      MODIFIED           
deepseek-r1:latest    6995872bfe4c    5.2 GB    About a minute ago    
llama3.2:latest       a80c4f17acd5    2.0 GB    9 minutes ago         
llama3.1:latest       46e0c10c039e    4.9 GB    25 minutes ago        
llama3:latest         365c0bd3c000    4.7 GB    31 minutes ago        
deepseek-r1:8b        6995872bfe4c    5.2 GB    32 minutes ago        
qwen3:30b             e50831eb2d91    18 GB     34 minutes ago        


In [ ]:
# ============================================================
# Mini-application IA financière — Solution 100% locale (Ollama)
# KPIs Yahoo Finance -> Prompt engineering (texte) -> Réponse locale
# AUCUN appel cloud (OpenAI non utilisé)
# ============================================================

import re
import numpy as np
import pandas as pd
import yfinance as yf
import ollama

# ------------------ Paramètres ------------------
TICKERS     = ["AAPL", "MSFT", "JPM", "XOM"]   # modifiez votre univers
START_DATE  = "2020-01-01"
END_DATE    = None
INTERVAL    = "1d"                             # "1d","1wk","1mo"
LLM_MODEL   = "llama3.1:latest"                      # ex. "llama3:8b", "mistral:7b-instruct", "qwen2.5:7b-instruct"
TEMP        = 0.2

# ------------------ 1) Import YF (Adj Close) ------------------
raw = yf.download(
    TICKERS,
    start=START_DATE,
    end=END_DATE,
    interval=INTERVAL,
    auto_adjust=False,     # on lit bien 'Adj Close'
    progress=False,
    group_by="column"
)

if isinstance(raw.columns, pd.MultiIndex):
    prices = raw["Adj Close"].copy()           # colonnes = tickers
else:
    prices = raw[["Adj Close"]].copy()         # colonnes = ["Adj Close"]
    if isinstance(TICKERS, list) and len(TICKERS) == 1:
        prices.columns = [TICKERS[0]]

prices = prices.dropna(how="all")
print("Aperçu des prix (Adj Close) :")
print(prices.head())

# ------------------ 2) KPIs sur les prix ------------------
# Rendements log
rets = np.log(prices).diff()

# KPIs : dernier cours, performance 1m (~21 j) / 3m (~63 j), vol 20j annualisée
win = 20
rows = []
for col in prices.columns:
    s = prices[col].dropna()
    r = rets[col].dropna()
    last_price = float(s.iloc[-1]) if len(s) else np.nan
    r_1m = float(np.exp(r.iloc[-21:].sum()) - 1) if len(r) >= 21 else np.nan
    r_3m = float(np.exp(r.iloc[-63:].sum()) - 1) if len(r) >= 63 else np.nan
    vol20 = float(r.iloc[-win:].std() * np.sqrt(252)) if len(r) >= win else np.nan
    rows.append({
        "ticker": col,
        "last_price": last_price,
        "return_1m": r_1m,
        "return_3m": r_3m,
        "vol20_annualized": vol20
    })

df_kpis = pd.DataFrame(rows)
print("\nKPIs bruts :")
print(df_kpis)

# Mise en forme pour un tableau lisible à injecter dans le prompt
k = df_kpis.copy()
k["last_price"]        = k["last_price"].map(lambda x: f"{x:,.2f} USD" if pd.notnull(x) else "NA")
k["return_1m"]         = k["return_1m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["return_3m"]         = k["return_3m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["vol20_annualized"]  = k["vol20_annualized"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")

DATA_DESC = "KPIs (Yahoo Finance):\n" + k.to_string(index=False)
print("\n=== Données injectées dans le prompt ===\n", DATA_DESC)

# ------------------ 3) Prompt template (sortie TEXTE) ------------------
PROMPT_TEMPLATE = """Vous etes un(e) analyste financier(ere) senior.
Contexte : {context}
Donnees fournies : {data_desc}

Taches (sortie texte) :
1) Resume (5 lignes max) clair et actionnable.
2) KPIs clefs (3 puces) au format : Nom - Valeur Unite/Devise (citer le chiffre exact).
3) Risques (2 puces) avec justification d'une ligne.
4) Actions proposees (3 puces) pour un decideur non specialiste.

Contraintes :
- Ton professionnel et concis ; eviter le jargon.
- Toujours citer les chiffres avec unite/devise (ex. M USD, %, etc.).
- Sortie en TEXTE : titres + listes a puces + commentaires et interprétations
"""

CONTEXT = "Portefeuille multi-secteurs, horizon court terme."
prompt = PROMPT_TEMPLATE.format(context=CONTEXT, data_desc=DATA_DESC)

# ------------------ 4) Génération locale avec Ollama ------------------
system_msg = (
    "Tu es un analyste financier. Reponds uniquement a partir des donnees fournies. "
    "Sortie en TEXTE (titres + puces + commentaires et interprétations) "
    "Cite clairement les chiffres avec leur unite/devise."
)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user",   "content": prompt}
]

print("\n=== Prompt (récapitulatif) ===\n")
print(prompt[:1000] + ("\n... (tronqué)" if len(prompt) > 1000 else ""))

resp = ollama.chat(
    model=LLM_MODEL,
    messages=messages,
    options={"temperature": TEMP}
)

print("\n=== Réponse du modèle (Ollama local) ===\n")
print(resp["message"]["content"])

# ============================================================
# Fin — Solution locale : aucun appel cloud, tout en local.
# ============================================================


KeyboardInterrupt: 